In [1]:
#pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [13]:
#pip install "tensorflow>=2.0.0"

Note: you may need to restart the kernel to use updated packages.


In [14]:
#pip install --upgrade tensorflow-hub

Note: you may need to restart the kernel to use updated packages.


In [1]:
import tensorflow as tf
import tensorflow_hub as hub
loaded_full_model = tf.keras.models.load_model("models-Full-image-set-mobilenetv2-adam.h5", custom_objects={"KerasLayer":hub.KerasLayer})

In [ ]:
import tkinter as tk
from PIL import ImageTk, Image
from tkinter import filedialog
import numpy as np
import tensorflow as tf
import os
import pandas as pd
labels_csv = pd.read_csv("labels.csv")
labels = labels_csv["breed"].to_numpy()
unique_breeds = np.unique(labels)

IMG_SIZE = 224
def process_image(image_path, img_size=IMG_SIZE):
  image = tf.io.read_file(image_path)
  image = tf.image.decode_jpeg(image, channels=3)
  image = tf.image.convert_image_dtype(image, tf.float32)
  image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])
  return image

def get_pred_label(prediction_probabilities):
  return unique_breeds[np.argmax(prediction_probabilities)]


BATCH_SIZE = 32
def create_data_batches(x, y=None, batch_size=BATCH_SIZE, valid_data=False, test_data=False):
  if test_data:
    print("Creating test data batches..")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x)))
    data_batch = data.map(process_image).batch(BATCH_SIZE)
    return data_batch
  elif valid_data:
    print("Creating validation data batches..")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x),
                                               tf.constant(y)))
    data_batch = data.map(get_image_label).batch(BATCH_SIZE)
    return data_batch 
  else:
    print("Creating training data batches..")
    data = tf.data.Dataset.from_tensor_slices((tf.constant(x),
                                              tf.constant(y)))
    data = data.shuffle(buffer_size=len(x))
    data = data.map(get_image_label)

    data_batch = data.batch(BATCH_SIZE)
  return data_batch

def load_img():
    global img, image_data
    for img_display in frame.winfo_children():
        img_display.destroy()

    image_data = filedialog.askopenfilename(initialdir="D:/ML_COURSE/Dog Breed Classification/dog_pics/", title="Choose an image",
                                       filetypes=(("all files", "*.*"), ("png files", "*.png")))
    basewidth = 350 # Processing image for dysplaying
    img = Image.open(image_data)
    wpercent = (basewidth / float(img.size[0]))
    hsize = int((float(img.size[1]) * float(wpercent)))
    img = img.resize((basewidth, hsize), Image.ANTIALIAS)
    img = ImageTk.PhotoImage(img)
    file_name = image_data.split('/')
    panel = tk.Label(frame, text= str(file_name[len(file_name)-1]).upper(), bg='white').pack()
    panel_image = tk.Label(frame, image=img).pack()

def classify():
    #original = Image.open(image_data)
    #original = original.resize((224, 224), Image.ANTIALIAS)
    custom_path = "D:/ML_COURSE/Dog Breed Classification/dog_pics/"
    custom_image_paths = [custom_path + fname for fname in os.listdir(custom_path)]
    custom_data = create_data_batches(custom_image_paths, test_data=True)
    custom_preds = loaded_full_model.predict(custom_data)
    custom_pred_labels = [get_pred_label(custom_preds[i]) for i in range(len(custom_preds))]
    label = custom_pred_labels[0]
    table = tk.Label(frame, text="The Breed of Dog is", font=("Comic Sans MS", 12), bg='white').pack()
    result = tk.Label(frame,text= str(label).upper(),  font=("Comic Sans MS", 20), bg='white').pack()
root = tk.Tk()
root.title('Dog Image Classifier')
root.resizable(False, False)
tit = tk.Label(root, text="Dog Image Classifier", padx=25, pady=6, font=("forte", 15)).pack()
canvas = tk.Canvas(root, height=500, width=500, bg='grey')
canvas.pack()
frame = tk.Frame(root, bg='white')
frame.place(relwidth=0.8, relheight=0.8, relx=0.1, rely=0.1)
chose_image = tk.Button(root, text='Choose Image',
                        padx=35, pady=10,
                        fg="white", bg="grey", command=load_img)
chose_image.pack(side=tk.LEFT)
class_image = tk.Button(root, text='Classify Image',
                        padx=35, pady=10,
                        fg="white", bg="grey", command=classify)
class_image.pack(side=tk.RIGHT)
root.mainloop()

Creating test data batches..
1/1 [==============================] - 1s 991ms/step
Creating test data batches..
1/1 [==============================] - 0s 114ms/step
